In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from copy import deepcopy
import random
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
from scipy import stats
import pgeocode
#from orion.contrib.envs import load_env
#load_env()
import boto3
#from orion.sources import S3Source
aws_bucket = 'kilimanjaro-prod-datalake'
s3 = boto3.client('s3')

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
trans = pd.read_csv('transaction_tidy.csv', index_col=0)

In [3]:
trans.head(1)

,salestransactionkey,salesordernumber,orderdate,ordertime,shippingdate,customerkey,shipcountry,postcode,itemnumber,pricetype,discountpercent,unitssold,grosssales,grossprofit,year,month,dayofweek,sales_type,gsales_perunit
0,2139607,S0016-8920840,2015-08-30,13:00:00,2015-08-30,220c0261086d7d2c254a9ba8d9339e6c15cb2792,United Kingdom,S61,90956,Clearance,0.0,1,58.16,15.9233,2015,8,Sunday,S0,58.16


In [4]:
trans.sort_values("orderdate", ascending=False, inplace = True)

In [5]:
trans.head(1)

,salestransactionkey,salesordernumber,orderdate,ordertime,shippingdate,customerkey,shipcountry,postcode,itemnumber,pricetype,discountpercent,unitssold,grosssales,grossprofit,year,month,dayofweek,sales_type,gsales_perunit
8591350,37013246,S0016-FA-22043819,2019-06-06,12:40:00,2019-06-06,ad629da3e798dddf583875098afdcd9d91e65b91,United Kingdom,SS8,116499,Clearance,0.0,1,21.52,3.175,2019,6,Thursday,S0,21.52


In [6]:
address = trans.drop_duplicates(subset =['customerkey'])
  
# displaying data
address.head(1)

,salestransactionkey,salesordernumber,orderdate,ordertime,shippingdate,customerkey,shipcountry,postcode,itemnumber,pricetype,discountpercent,unitssold,grosssales,grossprofit,year,month,dayofweek,sales_type,gsales_perunit
8591350,37013246,S0016-FA-22043819,2019-06-06,12:40:00,2019-06-06,ad629da3e798dddf583875098afdcd9d91e65b91,United Kingdom,SS8,116499,Clearance,0.0,1,21.52,3.175,2019,6,Thursday,S0,21.52


In [7]:
address = address[['customerkey','shipcountry','postcode']]

In [8]:
address.to_csv('address.csv')

In [30]:
prod_cust = pd.read_csv('Product_customer.csv', index_col=0)
prod_cust.head()

,customerkey,itemnumber,unitssold,brandcode,category,divisioncode,productgroup,orders,total_grosssales,total_items,total_unitssold,ASP,loyaltyaccount,gender
0,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2,NN,Junior,FOOTWEAR,Nylon Running,1,135.51,1,4,33.88,No,female
1,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2,NN,Junior,FOOTWEAR,Nylon Running,1,135.51,1,4,33.88,No,female
2,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,87139,2,EL,Mens,APPAREL,Woven Pants,1,17.45,2,2,8.72,No,female
3,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,89814,2,NONR,Miscellaneous,ACCESSORY,Non-retail Items,1,17.45,2,2,8.72,No,female
4,0000089aee56e015b7de440137bbd90baf2b6039,89813,5,NaN,Miscellaneous,ACCESSORY,Non-retail items,2,115.59,4,5,22.28,Yes,female


In [31]:
prod_cust[(prod_cust['customerkey']=='68961ae793932f6dc44b45f297980fb64dce2f9e')]

,customerkey,itemnumber,unitssold,brandcode,category,divisioncode,productgroup,orders,total_grosssales,total_items,total_unitssold,ASP,loyaltyaccount,gender


In [3]:
len(prod_cust)

7791933

In [3]:
prod_cust.isnull().sum()

customerkey               0
itemnumber                0
unitssold                 0
brandcode            908393
category                  0
divisioncode              0
productgroup              0
total_grosssales          0
total_items               0
orders                    0
ASP                       0
loyaltyaccount      1800273
gender              1998418
dtype: int64

In [4]:
(prod_cust['customerkey'].nunique())/2

1114594.5

In [4]:
prod_cust[:10]

,customerkey,itemnumber,unitssold,brandcode,category,divisioncode,productgroup,total_grosssales,total_items,orders,ASP,loyaltyaccount,gender
0,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2.0,NN,Junior,FOOTWEAR,Nylon Running,105.85,1,1,33.88,No,female
1,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2.0,NN,Junior,FOOTWEAR,Nylon Running,105.85,1,1,33.88,No,female
2,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,87139,2.0,EL,Mens,APPAREL,Woven Pants,17.45,2,1,8.72,No,female
3,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,89814,2.0,NONR,Miscellaneous,ACCESSORY,Non-retail Items,17.45,2,1,8.72,No,female
4,0000089aee56e015b7de440137bbd90baf2b6039,89813,5.0,NaN,Miscellaneous,ACCESSORY,Non-retail items,115.59,4,2,22.28,Yes,female
5,0000089aee56e015b7de440137bbd90baf2b6039,110940,5.0,AA,Mens,FOOTWEAR,Nylon Running,115.59,4,2,22.28,Yes,female
6,0000089aee56e015b7de440137bbd90baf2b6039,128777,5.0,EL,Mens,APPAREL,T-shirts,115.59,4,2,22.28,Yes,female
7,0000089aee56e015b7de440137bbd90baf2b6039,128801,5.0,EL,Mens,APPAREL,Light Jackets,115.59,4,2,22.28,Yes,female
8,000009008567d4740aec3d882e67fe4d2d74eb39,23372,3.0,VV,Womens,FOOTWEAR,Vulcanised,253.48,3,1,84.49,No,female
9,000009008567d4740aec3d882e67fe4d2d74eb39,47852,3.0,VV,Womens,FOOTWEAR,Vulcanised,253.48,3,1,84.49,No,female


In [3]:
start = 0 
category = []
for i in range (10000,len(prod_cust),10000):
    print(i)
    test = prod_cust[start:i]
    cat = test.groupby('customerkey')['category'].apply(lambda x: x.value_counts().index[0]).reset_index().values
    category.extend(cat)
    start = i 
test = prod_cust[start:len(product_cust)+1]
cat = test.groupby('customerkey')['category'].apply(lambda x: x.value_counts().index[0]).reset_index().values
category.extend(cat)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

NameError: name 'product_cust' is not defined

In [4]:
test = prod_cust[start:len(prod_cust)+1]
cat = test.groupby('customerkey')['category'].apply(lambda x: x.value_counts().index[0]).reset_index().values
category.extend(cat)

In [5]:
df = pd.DataFrame(category, columns=['customerkey', 'category']) 
df

,customerkey,category
0,0000010177de5b4048370276b036ff3ba06b7ae6,Junior
1,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,Mens
2,0000089aee56e015b7de440137bbd90baf2b6039,Mens
3,000009008567d4740aec3d882e67fe4d2d74eb39,Womens
4,0000167ab10ff470d5b1c2116b9aa33ddccc12bd,Infant
...,...,...
2229712,ffffdf2d6e78c31b7669022a96d6e8ab0f076404,Mens
2229713,ffffe0a8c1a504232cebca86d2be5e3f59cb9f92,Mens
2229714,ffffe0b8b360367113c4f1e3d88a9ceb14c031aa,Mens
2229715,fffff36d72733a953ace2a324b8b231502186204,Mens


In [6]:
df.to_csv('category.csv')

In [7]:
start = 0 
division = []
for i in range (10000,len(prod_cust),50000):
    print(i)
    test = prod_cust[start:i]
    div = test.groupby('customerkey')['divisioncode'].apply(lambda x: x.value_counts().index[0]).reset_index().values
    division.extend(div)
    start = i 
test = prod_cust[start:len(prod_cust)+1]
div = test.groupby('customerkey')['divisioncode'].apply(lambda x: x.value_counts().index[0]).reset_index().values
division.extend(div)

10000
60000
110000
160000
210000
260000
310000
360000
410000
460000
510000
560000
610000
660000
710000
760000
810000
860000
910000
960000
1010000
1060000
1110000
1160000
1210000
1260000
1310000
1360000
1410000
1460000
1510000
1560000
1610000
1660000
1710000
1760000
1810000
1860000
1910000
1960000
2010000
2060000
2110000
2160000
2210000
2260000
2310000
2360000
2410000
2460000
2510000
2560000
2610000
2660000
2710000
2760000
2810000
2860000
2910000
2960000
3010000
3060000
3110000
3160000
3210000
3260000
3310000
3360000
3410000
3460000
3510000
3560000
3610000
3660000
3710000
3760000
3810000
3860000
3910000
3960000
4010000
4060000
4110000
4160000
4210000
4260000
4310000
4360000
4410000
4460000
4510000
4560000
4610000
4660000
4710000
4760000
4810000
4860000
4910000
4960000
5010000
5060000
5110000
5160000
5210000
5260000
5310000
5360000
5410000
5460000
5510000
5560000
5610000
5660000
5710000
5760000
5810000
5860000
5910000
5960000
6010000
6060000
6110000
6160000
6210000
6260000
6310000
636000

In [8]:
div_df = pd.DataFrame(division, columns=['customerkey', 'divisioncode']) 
div_df.head(2)

,customerkey,divisioncode
0,0000010177de5b4048370276b036ff3ba06b7ae6,FOOTWEAR
1,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,APPAREL


In [9]:
len(div_df)

2229299

In [10]:
div_df = div_df.drop_duplicates(subset =['customerkey'])

In [11]:
div_df.to_csv('division.csv')

In [12]:
start = 0 
product = []
for i in range (10000,len(prod_cust),50000):
    print(i)
    test = prod_cust[start:i]
    prod = test.groupby('customerkey')['productgroup'].apply(lambda x: x.value_counts().index[0]).reset_index().values
    product.extend(prod)
    start = i 
test = prod_cust[start:len(prod_cust)+1]
prod = test.groupby('customerkey')['productgroup'].apply(lambda x: x.value_counts().index[0]).reset_index().values
product.extend(prod)

10000
60000
110000
160000
210000
260000
310000
360000
410000
460000
510000
560000
610000
660000
710000
760000
810000
860000
910000
960000
1010000
1060000
1110000
1160000
1210000
1260000
1310000
1360000
1410000
1460000
1510000
1560000
1610000
1660000
1710000
1760000
1810000
1860000
1910000
1960000
2010000
2060000
2110000
2160000
2210000
2260000
2310000
2360000
2410000
2460000
2510000
2560000
2610000
2660000
2710000
2760000
2810000
2860000
2910000
2960000
3010000
3060000
3110000
3160000
3210000
3260000
3310000
3360000
3410000
3460000
3510000
3560000
3610000
3660000
3710000
3760000
3810000
3860000
3910000
3960000
4010000
4060000
4110000
4160000
4210000
4260000
4310000
4360000
4410000
4460000
4510000
4560000
4610000
4660000
4710000
4760000
4810000
4860000
4910000
4960000
5010000
5060000
5110000
5160000
5210000
5260000
5310000
5360000
5410000
5460000
5510000
5560000
5610000
5660000
5710000
5760000
5810000
5860000
5910000
5960000
6010000
6060000
6110000
6160000
6210000
6260000
6310000
636000

In [14]:
prod_df = pd.DataFrame(product, columns=['customerkey', 'productgroup'])

In [15]:
prod_df.to_csv('product.csv')

In [5]:
prod_custb = prod_cust[(prod_cust['brandcode'].notnull())]

In [4]:
start = 0 
brand = []
for i in range (10000,100000,50000):
    print(i)
    test = prod_custb[start:i]
    br = test.groupby('customerkey')['brandcode'].apply(lambda x: x.value_counts().index[0]).reset_index().values
    brand.extend(br)
    start = i 
test = prod_custb[start:len(prod_custb)+1]
br = test.groupby('customerkey')['brandcode'].apply(lambda x: x.value_counts().index[0]).reset_index().values
brand.extend(br)

10000
60000


In [6]:
start = 0 
brand = []
for i in range (10000,len(prod_custb),50000):
    print(i)
    test = prod_custb[start:i]
    br = test.groupby('customerkey')['brandcode'].apply(lambda x: x.value_counts().index[0]).reset_index().values
    brand.extend(br)
    start = i 
test = prod_custb[start:len(prod_custb)+1]
prod = test.groupby('customerkey')['brandcode'].apply(lambda x: x.value_counts().index[0]).reset_index().values
brand.extend(br)

10000
60000
110000
160000
210000
260000
310000
360000
410000
460000
510000
560000
610000
660000
710000
760000
810000
860000
910000
960000
1010000
1060000
1110000
1160000
1210000
1260000
1310000
1360000
1410000
1460000
1510000
1560000
1610000
1660000
1710000
1760000
1810000
1860000
1910000
1960000
2010000
2060000
2110000
2160000
2210000
2260000
2310000
2360000
2410000
2460000
2510000
2560000
2610000
2660000
2710000
2760000
2810000
2860000
2910000
2960000
3010000
3060000
3110000
3160000
3210000
3260000
3310000
3360000
3410000
3460000
3510000
3560000
3610000
3660000
3710000
3760000
3810000
3860000
3910000
3960000
4010000
4060000
4110000
4160000
4210000
4260000
4310000
4360000
4410000
4460000
4510000
4560000
4610000
4660000
4710000
4760000
4810000
4860000
4910000
4960000
5010000
5060000
5110000
5160000
5210000
5260000
5310000
5360000
5410000
5460000
5510000
5560000
5610000
5660000
5710000
5760000
5810000
5860000
5910000
5960000
6010000
6060000
6110000
6160000
6210000
6260000
6310000
636000

In [7]:
brand_df = pd.DataFrame(brand, columns=['customerkey', 'brandcode'])

In [8]:
brand_df.to_csv('brandcode.csv')

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
cat = pd.read_csv('category.csv', index_col=0)
cat = cat.drop_duplicates(subset =['customerkey'])
cat.head(2)

,customerkey,category
0,0000010177de5b4048370276b036ff3ba06b7ae6,Junior
1,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,Mens


In [10]:
div = pd.read_csv('division.csv', index_col=0)
div = div.drop_duplicates(subset =['customerkey'])
div.head(2)

,customerkey,divisioncode
0,0000010177de5b4048370276b036ff3ba06b7ae6,FOOTWEAR
1,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,APPAREL


In [11]:
prod = pd.read_csv('product.csv', index_col=0)
prod = prod.drop_duplicates(subset =['customerkey'])
prod.head(2)

,customerkey,productgroup
0,0000010177de5b4048370276b036ff3ba06b7ae6,Nylon Running
1,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,Non-retail Items


In [13]:
brand = pd.read_csv('brandcode.csv', index_col=0)
brand = brand.drop_duplicates(subset =['customerkey'])
brand.head(2)

,customerkey,brandcode
0,0000010177de5b4048370276b036ff3ba06b7ae6,NN
1,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,NONR


In [18]:
full_df = cat.merge(div, how='left', on=['customerkey'])

In [19]:
full_df = full_df.merge(prod, how='left', on=['customerkey'])

In [20]:
full_df = full_df.merge(brand, how='left', on=['customerkey'])

In [21]:
full_df.head(2)

,customerkey,category,divisioncode,productgroup,brandcode
0,0000010177de5b4048370276b036ff3ba06b7ae6,Junior,FOOTWEAR,Nylon Running,NN
1,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,Mens,APPAREL,Non-retail Items,NONR


In [22]:
full_df.to_csv('cust_prod.csv')

In [73]:
cust_summary = pd.read_csv('cust_summary.csv', index_col=0)

In [74]:
cust_summary.head(1)

,customerkey,orders,total_unitssold,total_grosssales,ASP,loyaltyaccount,gender
0,fb484d6a54da0d56910e2df0064c18c722801a66,256,603,35645.08,61.44,Yes,female


In [75]:
cust_summary = cust_summary.merge(full_df, how='left', on=['customerkey'])

In [76]:
cust_summary.head(1)

,customerkey,orders,total_unitssold,total_grosssales,ASP,loyaltyaccount,gender,category,divisioncode,productgroup,brandcode
0,fb484d6a54da0d56910e2df0064c18c722801a66,256,603,35645.08,61.44,Yes,female,Junior,FOOTWEAR,Basketball,AA


In [79]:
cust_remove = cust_summary[(cust_summary['category'].isnull())]
cust_remove.head(1)

,customerkey,orders,total_unitssold,total_grosssales,ASP,loyaltyaccount,gender,category,divisioncode,productgroup,brandcode
69699,68961ae793932f6dc44b45f297980fb64dce2f9e,5,14,853.66,63.59,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
cust_remove = cust_remove['customerkey'].values
cust_remove

array(['68961ae793932f6dc44b45f297980fb64dce2f9e',
       'eee924a7f86094141baa9956055e1d693f1e6816',
       'b36df96652c0aa6b1e4505ae167e5210c12a8c74', ...,
       '62128951063bbe416282784ba6ac60b0df79bc3f',
       '62128283f4d3a5146187d3bd4f6fb504bf0d6163',
       '62125d2732919b04d3734b8597b66ef329613829'], dtype=object)

In [81]:
cust_summary = cust_summary[~cust_summary.customerkey.isin(cust_remove)]

In [82]:
cust_summary.isnull().sum()

customerkey              0
orders                   0
total_unitssold          0
total_grosssales         0
ASP                      0
loyaltyaccount      658638
gender              735588
category                 0
divisioncode             0
productgroup             0
brandcode            37519
dtype: int64

In [83]:
loyal_check = cust_summary[(cust_summary['loyaltyaccount'].isnull())]
loyal_check.head(1)

,customerkey,orders,total_unitssold,total_grosssales,ASP,loyaltyaccount,gender,category,divisioncode,productgroup,brandcode
4,efef72470d6b8bc80f22afd9192a39f26357095f,135,263,16536.17,68.6,NaN,NaN,Junior,FOOTWEAR,Basketball,NN


In [84]:
loyal_remove = loyal_check['customerkey'].values
loyal_remove

array(['efef72470d6b8bc80f22afd9192a39f26357095f',
       '4e28db0878ca206a0d20b16fe185808459aa4638',
       '851a7a0aa3e2d9dfe1a626c5cea346cf4fe54b1e', ...,
       '620ff4deefb419efcbc0d79e39ec7aa4e00ed8ee',
       '620fd5491e0adf341547f1575b406647234fa96e',
       '620fae4b0616fdc60a596a37bbb2ae362176943b'], dtype=object)

In [85]:
cust_summary = cust_summary[~cust_summary.customerkey.isin(loyal_remove)]

In [86]:
cust_summary.isnull().sum()

customerkey             0
orders                  0
total_unitssold         0
total_grosssales        0
ASP                     0
loyaltyaccount          0
gender              76950
category                0
divisioncode            0
productgroup            0
brandcode           28891
dtype: int64

In [87]:
cust_summary['gender'] = cust_summary['gender'].fillna('unknown')

In [88]:
gend_check = cust_summary[(cust_summary['gender'].isnull())]
gend_check.head(1)

,customerkey,orders,total_unitssold,total_grosssales,ASP,loyaltyaccount,gender,category,divisioncode,productgroup,brandcode


In [89]:
len(cust_summary)

2204873

In [90]:
cust_summary.isnull().sum()

customerkey             0
orders                  0
total_unitssold         0
total_grosssales        0
ASP                     0
loyaltyaccount          0
gender                  0
category                0
divisioncode            0
productgroup            0
brandcode           28891
dtype: int64

In [91]:
cust_summary['brandcode'] = cust_summary['brandcode'].fillna('none')

In [155]:
cust_summary.to_csv('cust_summary.csv')

NameError: name 'cust_summary' is not defined

In [93]:
address = pd.read_csv('address.csv', index_col=0)
address.head(1)

,customerkey,shipcountry,postcode
8591350,ad629da3e798dddf583875098afdcd9d91e65b91,United Kingdom,SS8


In [94]:
cust_summary1 = cust_summary.merge(address, how='left', on='customerkey')

In [95]:
cust_summary1.isnull().sum()

customerkey           0
orders                0
total_unitssold       0
total_grosssales      0
ASP                   0
loyaltyaccount        0
gender                0
category              0
divisioncode          0
productgroup          0
brandcode             0
shipcountry           0
postcode            445
dtype: int64

In [97]:
cust_summary1[cust_summary1['postcode'].isnull()]

,customerkey,orders,total_unitssold,total_grosssales,ASP,loyaltyaccount,gender,category,divisioncode,productgroup,brandcode,shipcountry,postcode
1974,1c137fb3b669fdc6437d8d43d20188c992f60d8c,14,46,2296.49,48.51,No,male,Womens,FOOTWEAR,Nylon Running,AA,Hong Kong,NaN
21036,b25de5201f62d7b1c1f6b10e14d868b0d37a2f7d,7,17,598.38,36.28,Yes,female,Mens,APPAREL,Nylon Running,NONR,Ireland,NaN
28795,403b6d8e9f2393243ed8dee7af331e7120a0e68b,6,12,592.16,49.35,No,male,Mens,FOOTWEAR,Tennis,AA,Ireland,NaN
38236,b7ae2a39d7e05880862973d12a05bf814b9bdd14,6,18,895.70,50.25,No,male,Womens,FOOTWEAR,Nylon Running,AA,Hong Kong,NaN
76463,7c1241400c6211c46ce7035fcc51475820748d33,4,13,532.24,40.03,Yes,male,Mens,APPAREL,Nylon Running,NN,Ireland,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2183417,64c5a8c23da5649898c4b627fff2dc4478c4e5a7,1,9,454.19,50.47,No,male,Junior,FOOTWEAR,Basketball,AA,Hong Kong,NaN
2183418,64c5a8c23da5649898c4b627fff2dc4478c4e5a7,1,9,454.19,50.47,No,male,Junior,FOOTWEAR,Basketball,AA,Hong Kong,NaN
2188906,647c9f39fc3030d7e41ddf6232413e24767bb9b3,1,3,141.79,47.26,No,female,Womens,FOOTWEAR,Nylon Running,RR,Hong Kong,NaN
2191619,613c12e295a54cf47fd3f46776504c3ba2445d1f,1,4,107.74,26.94,No,female,Mens,FOOTWEAR,Basketball,AA,Hong Kong,NaN


In [99]:
cust_summary1['postcode'] = cust_summary1['postcode'].fillna('International')

In [100]:
ust_summary1 = cust_summary1[['customerkey','orders','total_unitssold','total_grosssales','ASP','loyaltyaccount','gender', 'shcipcountry','postcode', 'category','divisioncode','productgroup','brandcode']]

In [103]:
order_lag = pd.read_csv('Order_lag.csv')
order_lag.head(2)

,customerkey,days_bw_orders
0,0000089aee56e015b7de440137bbd90baf2b6039,107
1,0000167ab10ff470d5b1c2116b9aa33ddccc12bd,121


In [104]:
cust_summary1 = cust_summary1.merge(order_lag, how='left', on='customerkey')

In [106]:
cust_summary1 = cust_summary1[['customerkey','orders','total_unitssold','total_grosssales','days_bw_orders','ASP','loyaltyaccount','gender', 'shipcountry','postcode', 'category','divisioncode','productgroup','brandcode']]

In [107]:
cust_summary1.head(2)

,customerkey,orders,total_unitssold,total_grosssales,days_bw_orders,ASP,loyaltyaccount,gender,shipcountry,postcode,category,divisioncode,productgroup,brandcode
0,fb484d6a54da0d56910e2df0064c18c722801a66,256,603,35645.08,4.0,61.44,Yes,female,United Kingdom,TW9,Junior,FOOTWEAR,Basketball,AA
1,649cca818138ed8152825855e0538ed7360bd55a,152,400,18320.26,9.0,51.97,Yes,female,United Kingdom,DL5,Mens,APPAREL,Nylon Running,NN


In [110]:
cust_summary1['days_bw_orders'].isnull().sum()

1625249

In [111]:
cust_summary1['days_bw_orders'] = cust_summary1['days_bw_orders'].fillna('1825')

In [113]:
cust_summary1['days_bw_orders']  = cust_summary1['days_bw_orders'].astype('int64')

In [114]:
cust_summary1.to_csv('cust_summary.csv')

In [3]:
cust_summary1 = pd.read_csv('cust_summary.csv', index_col=0)

In [4]:
cust_summary1.head()

,customerkey,orders,total_unitssold,total_grosssales,days_bw_orders,ASP,loyaltyaccount,gender,shipcountry,postcode,category,divisioncode,productgroup,brandcode,latitude,longitude,areacode
0,fb484d6a54da0d56910e2df0064c18c722801a66,256,603,35645.08,4,61.44,Yes,female,United Kingdom,TW9,Junior,FOOTWEAR,Basketball,AA,51.4728,-0.296367,UK
1,649cca818138ed8152825855e0538ed7360bd55a,152,400,18320.26,9,51.97,Yes,female,United Kingdom,DL5,Mens,APPAREL,Nylon Running,NN,54.6000,-1.587800,UK
2,559aec727d6d73e55e476a34a374f1fb26dc06cf,152,401,16730.55,7,44.44,Yes,female,United Kingdom,LL18,Mens,FOOTWEAR,Vulcanised,NN,53.2910,-3.454820,UK
3,35d17f77612459a698d0179b7525c6eedacc6fd6,144,469,15046.12,8,37.51,Yes,female,United Kingdom,N19,Mens,APPAREL,T-shirts,KWD,51.5649,-0.135100,UK
4,67f9aeeef82eefa706b6886df0b790992e3b5a61,114,225,11631.89,11,63.97,Yes,male,United Kingdom,WA1,Womens,FOOTWEAR,Nylon Running,AA,53.3925,-2.561000,UK


In [7]:
cust_summary1[cust_summary1['latitude'].isnull()]

,customerkey,orders,total_unitssold,total_grosssales,days_bw_orders,ASP,loyaltyaccount,gender,shipcountry,postcode,category,divisioncode,productgroup,brandcode,latitude,longitude,areacode
9,0654e52ec663b2cffe038c823dee4f4ab0cdda64,96,256,10280.19,8,39.21,No,male,Japan,270-,Mens,APPAREL,Nylon Running,NN,NaN,NaN,JP
12,ab64e057b3895e5b965f83e13cfde07ca10b344d,89,520,12239.33,10,24.13,Yes,female,New Zealand,4412,Nursery,FOOTWEAR,Nylon Running,NN,NaN,NaN,NZ
14,315b34aef4b5f0120135792e1d333a88313d088a,82,176,6848.37,10,38.57,Yes,male,Singapore,1440,Junior,FOOTWEAR,Tennis,AA,NaN,NaN,NaN
16,f91a83c41bc2e316c74fe6714cb75a1aba4eed85,76,329,12266.29,16,37.60,Yes,male,Japan,2570,Mens,APPAREL,Caps,NN,NaN,NaN,JP
18,2c0935304b5a923a84bbaf65a076372c42a8f1f5,67,184,6987.22,3,37.76,Yes,female,Unknown,2993,Mens,APPAREL,Nylon Running,AA,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2204629,6219e8f5b005b19796bedb609423a5567d40831a,1,4,154.01,1825,38.50,No,unknown,Ireland,13,Miscellaneous,ACCESSORY,Non-retail Items,NONR,NaN,NaN,IE
2204630,6219e8f5b005b19796bedb609423a5567d40831a,1,4,154.01,1825,38.50,No,unknown,Ireland,13,Miscellaneous,ACCESSORY,Non-retail Items,NONR,NaN,NaN,IE
2204639,621aea17db649392cfcd76a9d6c90a412c61379b,1,2,95.73,1825,47.86,No,female,Hong Kong,0000,Womens,FOOTWEAR,Vintage,AA,NaN,NaN,NaN
2204731,6209d2226c36dad596b526c64fbfd98ed331d127,1,10,484.97,1825,48.50,No,male,Hong Kong,NIL,Mens,FOOTWEAR,Nylon Running,AA,NaN,NaN,NaN


In [14]:
cust_summary1.isnull().sum()

customerkey             0
orders                  0
total_unitssold         0
total_grosssales        0
days_bw_orders          0
ASP                     0
loyaltyaccount          0
gender                  0
shipcountry             0
postcode                0
category                0
divisioncode            0
productgroup            0
brandcode               0
latitude            79481
longitude           79481
areacode             9685
dtype: int64

In [15]:
pc = cust_summary1['postcode'].values
nomi = pgeocode.Nominatim('GB')

In [16]:
nomi = pgeocode.Nominatim('GB')

In [17]:
location = nomi.query_postal_code(pc)
location.head(1)

,postal_code,country_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,TW9,GB,"Kew Gardens, Richmond, Kew",England,ENG,Greater London,11609024,NaN,NaN,51.4728,-0.296367,4.0


In [18]:
cust_summary1['latitude']= location['latitude']
cust_summary1['longitude']= location['longitude']

In [19]:
cust_summary1.head(4)

,customerkey,orders,total_unitssold,total_grosssales,days_bw_orders,ASP,loyaltyaccount,gender,shipcountry,postcode,category,divisioncode,productgroup,brandcode,latitude,longitude,areacode
0,fb484d6a54da0d56910e2df0064c18c722801a66,256,603,35645.08,4,61.44,Yes,female,United Kingdom,TW9,Junior,FOOTWEAR,Basketball,AA,51.4728,-0.296367,UK
1,649cca818138ed8152825855e0538ed7360bd55a,152,400,18320.26,9,51.97,Yes,female,United Kingdom,DL5,Mens,APPAREL,Nylon Running,NN,54.6000,-1.587800,UK
2,559aec727d6d73e55e476a34a374f1fb26dc06cf,152,401,16730.55,7,44.44,Yes,female,United Kingdom,LL18,Mens,FOOTWEAR,Vulcanised,NN,53.2910,-3.454820,UK
3,35d17f77612459a698d0179b7525c6eedacc6fd6,144,469,15046.12,8,37.51,Yes,female,United Kingdom,N19,Mens,APPAREL,T-shirts,KWD,51.5649,-0.135100,UK


In [20]:
location.isnull().sum()

postal_code             0
country_code       123235
place_name         123235
state_name         126647
state_code         126647
county_name        145617
county_code        231134
community_name    2204804
community_code    2204804
latitude           123235
longitude          123235
accuracy           123235
dtype: int64

In [21]:
cust_summary1['new_postcode'] = cust_summary1['postcode'].str[:3]

In [22]:
pc1 = cust_summary1['new_postcode'].values

In [23]:
location2 = nomi.query_postal_code(pc1)
location2.head(1)

,postal_code,country_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,TW9,GB,"Kew Gardens, Richmond, Kew",England,ENG,Greater London,11609024,NaN,NaN,51.4728,-0.296367,4.0


In [27]:
cust_summary1['new_latitude']= location2['latitude']
cust_summary1['new_longitude']= location2['longitude']

In [28]:
cust_summary1['latitude'] = cust_summary1['latitude'].fillna(cust_summary1['new_latitude'])
cust_summary1['longitude'] = cust_summary1['longitude'].fillna(cust_summary1['new_longitude'])

In [29]:
cust_summary1['latitude'].isnull().sum()

79481

In [30]:
cust_summary1 = cust_summary1.drop(columns=['new_postcode', 'new_latitude', 'new_longitude'])

In [32]:
na_pc = cust_summary1[cust_summary1['latitude'].isnull() & (cust_summary1['postcode']!= 'unknown')]
na_pc.head(5)

,customerkey,orders,total_unitssold,total_grosssales,days_bw_orders,ASP,loyaltyaccount,gender,shipcountry,postcode,category,divisioncode,productgroup,brandcode,latitude,longitude,areacode
9,0654e52ec663b2cffe038c823dee4f4ab0cdda64,96,256,10280.19,8,39.21,No,male,Japan,270-,Mens,APPAREL,Nylon Running,NN,NaN,NaN,JP
12,ab64e057b3895e5b965f83e13cfde07ca10b344d,89,520,12239.33,10,24.13,Yes,female,New Zealand,4412,Nursery,FOOTWEAR,Nylon Running,NN,NaN,NaN,NZ
14,315b34aef4b5f0120135792e1d333a88313d088a,82,176,6848.37,10,38.57,Yes,male,Singapore,1440,Junior,FOOTWEAR,Tennis,AA,NaN,NaN,NaN
16,f91a83c41bc2e316c74fe6714cb75a1aba4eed85,76,329,12266.29,16,37.60,Yes,male,Japan,2570,Mens,APPAREL,Caps,NN,NaN,NaN,JP
18,2c0935304b5a923a84bbaf65a076372c42a8f1f5,67,184,6987.22,3,37.76,Yes,female,Unknown,2993,Mens,APPAREL,Nylon Running,AA,NaN,NaN,NaN


In [33]:
d = {'Japan':'JP', 'New Zealand':'NZ','Ireland':'IE', 'Netherlands':'NL','Slovakia':'SK','Malta':'MT',
     'United States':'US','France':'FR',
     'Guernsey':'GG','Poland':'PL', 'Slovenia':'SI','Ukraine':'UA','Romania':'RO','Russia':'RU','Sweden':'SE',
     'Germany':'DE','Australia':'AU',
     'Belgium':'BE','Austria':'AT','Spain':'ES','Belarus':'BY','Portugal':'PT',
     'Latvia':'LV','Czechia':'CZ','Canada':'CA','Denmark':'DE','Iceland':'IS','Bulgaria':'BG','Isle of Man':'IM',
     'Lithuania':'LT','Croatia':'HR','Norway':'NO','Hungary':'HU',
     'Luxembourg':'LU','Greenland':'GL','Finland':'FI','Switzerland':'CH','San Marino':'SM','Italy':'IT','Turkey':'TR',
     'Macedonia':'MK','Jersey':'JE','Liechtenstein':'LI', 'United Kingdom':'UK'}

In [34]:
cust_summary1['areacode'] = cust_summary1['shipcountry'].map(d)

In [35]:
cust_summary1.head(1)

,customerkey,orders,total_unitssold,total_grosssales,days_bw_orders,ASP,loyaltyaccount,gender,shipcountry,postcode,category,divisioncode,productgroup,brandcode,latitude,longitude,areacode
0,fb484d6a54da0d56910e2df0064c18c722801a66,256,603,35645.08,4,61.44,Yes,female,United Kingdom,TW9,Junior,FOOTWEAR,Basketball,AA,51.4728,-0.296367,UK


In [36]:
notnull_pc = cust_summary1[cust_summary1['latitude'].notnull()]
null_pc = cust_summary1[cust_summary1['latitude'].isnull()]

In [37]:
null_pc = cust_summary1[cust_summary1['latitude'].isnull()]

In [38]:
country_codes = list(d.values())

In [40]:
for c in country_codes:
    nomi = pgeocode.Nominatim(c)
    print(c)
    test = deepcopy(null_pc[null_pc['areacode']== c])
    pc_test = test['postcode'].values
    location2 = nomi.query_postal_code(pc_test)
    test['latitude']= location2['latitude']
    test['longitude']= location2['longitude']
    notnull_pc = pd.concat([notnull_pc, test])

JP
NZ
IE
NL
SK
MT
US
FR
GG
PL
SI
UA
RO
RU
SE
DE
AU
BE
AT
ES
BY
PT
LV
CZ
CA
DE
IS
BG
IM
LT
HR
NO
HU
LU
GL
FI
CH
SM
IT
TR
MK
JE
LI


ValueError: country=UK is not a known country code. See the README for a list of supported countries

In [67]:
cust_summary2[(cust_summary2['latitude'].isnull())& (cust_summary2['shipcountry']=='Germany')]

,customerkey,orders,total_unitssold,total_grosssales,days_bw_orders,ASP,loyaltyaccount,gender,shipcountry,postcode,category,divisioncode,productgroup,brandcode,latitude,longitude,areacode


In [42]:
ncountry = notnull_pc[notnull_pc['latitude'].isnull()]

In [43]:
ncountry['shipcountry'].unique()

array(['Japan', 'New Zealand', 'Ireland', 'Netherlands', 'Slovakia',
       'Malta', 'United States', 'France', 'Guernsey', 'Poland',
       'Slovenia', 'Ukraine', 'Romania', 'Russia', 'Sweden', 'Germany',
       'Denmark', 'Australia', 'Belgium', 'Austria', 'Spain', 'Belarus',
       'Portugal', 'Latvia', 'Czechia', 'Canada', 'Iceland', 'Bulgaria',
       'Isle of Man', 'Lithuania', 'Croatia', 'Norway', 'Hungary',
       'Luxembourg', 'Greenland', 'Finland', 'Switzerland', 'San Marino',
       'Italy', 'Turkey', 'Macedonia', 'Jersey', 'Liechtenstein'],
      dtype=object)

In [60]:
cust_summary2 = deepcopy(notnull_pc)

In [61]:
cust_summary2["latitude"] = cust_summary2.groupby("shipcountry").transform(lambda x: x.fillna(x.mean()))['latitude']

In [63]:
cust_summary2["longitude"] = cust_summary2.groupby("shipcountry").transform(lambda x: x.fillna(x.mean()))['longitude']

In [64]:
cust_summary2['latitude'].isnull().sum()

10228

In [65]:
cust_summary2.head(10)

,customerkey,orders,total_unitssold,total_grosssales,days_bw_orders,ASP,loyaltyaccount,gender,shipcountry,postcode,category,divisioncode,productgroup,brandcode,latitude,longitude,areacode
0,fb484d6a54da0d56910e2df0064c18c722801a66,256,603,35645.08,4,61.44,Yes,female,United Kingdom,TW9,Junior,FOOTWEAR,Basketball,AA,51.4728,-0.296367,UK
1,649cca818138ed8152825855e0538ed7360bd55a,152,400,18320.26,9,51.97,Yes,female,United Kingdom,DL5,Mens,APPAREL,Nylon Running,NN,54.6000,-1.587800,UK
2,559aec727d6d73e55e476a34a374f1fb26dc06cf,152,401,16730.55,7,44.44,Yes,female,United Kingdom,LL18,Mens,FOOTWEAR,Vulcanised,NN,53.2910,-3.454820,UK
3,35d17f77612459a698d0179b7525c6eedacc6fd6,144,469,15046.12,8,37.51,Yes,female,United Kingdom,N19,Mens,APPAREL,T-shirts,KWD,51.5649,-0.135100,UK
4,67f9aeeef82eefa706b6886df0b790992e3b5a61,114,225,11631.89,11,63.97,Yes,male,United Kingdom,WA1,Womens,FOOTWEAR,Nylon Running,AA,53.3925,-2.561000,UK
5,17c97a02c3f208c0f66365adafef6dd0219c3df8,113,460,15273.28,10,37.88,Yes,male,United Kingdom,KT3,Mens,APPAREL,Caps,NN,51.3992,-0.255600,UK
6,17c97a02c3f208c0f66365adafef6dd0219c3df8,113,460,15273.28,10,37.88,Yes,male,United Kingdom,KT3,Mens,APPAREL,Caps,NN,51.3992,-0.255600,UK
7,d9e087857a5fc93bb7e2cad9855125b6c3b8db19,107,343,15693.95,4,46.59,Yes,female,United Kingdom,CT11,Junior,FOOTWEAR,Basketball,NN,51.3358,1.415500,UK
8,5f6ffb95c67d0c380d674d73647e9fe0169204c0,101,189,9826.23,9,57.08,Yes,female,United Kingdom,AL4,Womens,FOOTWEAR,Basketball,AA,51.7809,-0.284283,UK
10,e54482736219d03154972a9362a4e489d5f897d4,94,123,9791.11,9,83.27,Yes,male,United Kingdom,BT34,Mens,FOOTWEAR,Nylon Running,NN,54.2500,-6.121125,UK


In [68]:
nas= cust_summary2[cust_summary2['latitude'].isnull()]
nas

,customerkey,orders,total_unitssold,total_grosssales,days_bw_orders,ASP,loyaltyaccount,gender,shipcountry,postcode,category,divisioncode,productgroup,brandcode,latitude,longitude,areacode
9,0654e52ec663b2cffe038c823dee4f4ab0cdda64,96,256,10280.19,8,39.21,No,male,Japan,270-,Mens,APPAREL,Nylon Running,NN,NaN,NaN,JP
16,f91a83c41bc2e316c74fe6714cb75a1aba4eed85,76,329,12266.29,16,37.60,Yes,male,Japan,2570,Mens,APPAREL,Caps,NN,NaN,NaN,JP
20,5c1f27f834b4a29304ac48d1924bcfad645b2f9d,62,248,8093.59,3,33.19,No,male,Japan,2750,Mens,APPAREL,Sandals,NN,NaN,NaN,JP
32,3a6960221b085527f80bd8a08ac5656ddecaa9c3,48,220,7580.40,23,34.81,Yes,female,Japan,2730,Miscellaneous,ACCESSORY,Caps,AA,NaN,NaN,JP
33,3a6960221b085527f80bd8a08ac5656ddecaa9c3,48,220,7580.40,23,34.81,Yes,female,Japan,2730,Miscellaneous,ACCESSORY,Caps,AA,NaN,NaN,JP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373830,2a37d8573fc3805864d5c4422d2655306f85dcb1,1,2,39.42,1825,19.71,No,male,Macedonia,1000,Mens,APPAREL,T-shirts,GOD,NaN,NaN,MK
1732205,194e5795985f2a9c049b7ebef18b8f04e9a52d09,1,2,108.31,1825,54.15,No,female,Macedonia,1000,Mens,FOOTWEAR,Vulcanised,CO,NaN,NaN,MK
1799494,70cf1769184408f76391db84391e745682506727,1,2,53.82,1825,26.91,Yes,female,Macedonia,1000,Mens,FOOTWEAR,Nylon Running,AA,NaN,NaN,MK
2088670,68092e13fe767cbd15c77e07523e2194765a10cf,1,2,106.54,1825,53.27,No,male,Macedonia,1300,Womens,FOOTWEAR,Nylon Running,NN,NaN,NaN,MK


In [69]:
countries = nas['shipcountry'].unique()
countries

array(['Japan', 'Slovakia', 'Malta', 'Slovenia', 'Denmark', 'Belgium',
       'Belarus', 'Portugal', 'Latvia', 'Czechia', 'Iceland', 'Croatia',
       'Norway', 'Hungary', 'Greenland', 'Finland', 'San Marino', 'Italy',
       'Turkey', 'Macedonia', 'Liechtenstein'], dtype=object)

In [54]:
latitude= [36.2048,48.10, 35.5, 46.04,55.41, 50.51, 53.52, 38.42, 56.53, 50.05, 64.10, 45.50, 59.55, 47.29, 64.10, 60.15, 43.55, 41.54, 39.57, 42.01, 47.08]

In [55]:
longitude = [138.2529, 17.07, 14.31, 14.33, 12.24, 04.21, 27.3, 09.10, 24.08, 14.22, 21.57, 15.58, 10.45, 19.05, 51.35, 25.03, 12.30, 12.29, 32.54, 21.26, 9.31]

In [71]:
countries_nas = pd.DataFrame(
    {'Country': countries,
     'Latitude': latitude,
     'Longitude': longitude
    })

In [72]:
countries_nas = countries_nas.set_index('Country')
countries_nas

,Latitude,Longitude
Country,,
Japan,36.2048,138.2529
Slovakia,48.1000,17.0700
Malta,35.5000,14.3100
Slovenia,46.0400,14.3300
Denmark,55.4100,12.2400
Belgium,50.5100,4.2100
Belarus,53.5200,27.3000
Portugal,38.4200,9.1000
Latvia,56.5300,24.0800


In [73]:
countries_nas.loc['Latvia',['Latitude']][0]

56.53

In [74]:
notnull_pc = cust_summary2[cust_summary2['latitude'].notnull()]
null_pc = cust_summary2[cust_summary2['latitude'].isnull()]

In [75]:
for c in countries:
    test = deepcopy(null_pc[null_pc['shipcountry']== c])
    test['latitude'] = test['latitude'].fillna(countries_nas.loc[c,['Latitude']][0])
    test['longitude'] = test['longitude'].fillna(countries_nas.loc[c,['Longitude']][0])
    notnull_pc = pd.concat([notnull_pc, test])

In [234]:
notnull_pc.tail()

,customerkey,orders,total_unitssold,total_grosssales,days_bw_orders,ASP,loyaltyaccount,gender,shipcountry,postcode,category,divisioncode,productgroup,brandcode,latitude,longitude,areacode
1373830,2a37d8573fc3805864d5c4422d2655306f85dcb1,1,2,39.42,1825,19.71,No,male,Macedonia,1000,Mens,APPAREL,T-shirts,GOD,42.01,21.26,MK
1732205,194e5795985f2a9c049b7ebef18b8f04e9a52d09,1,2,108.31,1825,54.15,No,female,Macedonia,1000,Mens,FOOTWEAR,Vulcanised,CO,42.01,21.26,MK
1799494,70cf1769184408f76391db84391e745682506727,1,2,53.82,1825,26.91,Yes,female,Macedonia,1000,Mens,FOOTWEAR,Nylon Running,AA,42.01,21.26,MK
2088670,68092e13fe767cbd15c77e07523e2194765a10cf,1,2,106.54,1825,53.27,No,male,Macedonia,1300,Womens,FOOTWEAR,Nylon Running,NN,42.01,21.26,MK
1835172,772e4beeb6d36cd3882e31a1fa327e352ab33f11,1,5,140.41,1825,28.08,No,male,Liechtenstein,9497,Mens,APPAREL,T-shirts,GFN,47.08,9.31,LI


In [76]:
notnull_pc.isnull().sum()

customerkey           0
orders                0
total_unitssold       0
total_grosssales      0
days_bw_orders        0
ASP                   0
loyaltyaccount        0
gender                0
shipcountry           0
postcode              0
category              0
divisioncode          0
productgroup          0
brandcode             0
latitude              0
longitude             0
areacode            295
dtype: int64

In [77]:
custsummary1 = deepcopy(notnull_pc)

In [78]:
custsummary1 = custsummary1.drop(columns=['areacode'])

In [79]:
custsummary1.to_csv('cust_summary.csv')